__1 - Importing needed dependencies__

In [1]:
import tensorflow as tf
import json
import keras
import os
import sys
import re
import copy
import numpy as np
import pandas as pd
import random
import gensim
from copy import deepcopy
from gensim.models import Word2Vec
from sklearn.utils import shuffle
from keras_transformer import get_model, decode
from tensorflow.keras.preprocessing.text import text_to_word_sequence
from keras.models import load_model

Using TensorFlow backend.


__2 - Declaring global variables__

In [2]:
PREPROCESS = "CONTINOUS" # CONTINOUS or DOTS
EPOCHS = 100
SENTENCES_MAX_LENGTH = 200
BATCH_SIZE = 16
EMBEDDING_DIM = 128
HIDDEN_DIM = 1024
NUM_TRAIN_PLOTS = 147
ENCODERS = 1
DECODERS = 1
DROPOUT_RATE = 0.1
HEADS_ATTENTION = 8
ACTIVATION_FUNCTION = 'relu'

__3 - Try read data__

In [3]:
def clean(text):
    '''
    '''
    text = text.strip()
    text = text.replace("ain't", "am not")
    text = text.replace("aren't", "are not")
    text = text.replace("can't", "cannot")
    text = text.replace("can't've", "cannot have")
    text = text.replace("'cause", "because")
    text = text.replace("could've", "could have")
    text = text.replace("couldn't", "could not")
    text = text.replace("couldn't've", "could not have")
    text = text.replace("should've", "should have")
    text = text.replace("should't", "should not")
    text = text.replace("should't've", "should not have")
    text = text.replace("would've", "would have")
    text = text.replace("would't", "would not")
    text = text.replace("would't've", "would not have")
    text = text.replace("didn't", "did not")
    text = text.replace("doesn't", "does not")
    text = text.replace("don't", "do not")
    text = text.replace("hadn't", "had not")
    text = text.replace("hadn't've", "had not have")
    text = text.replace("hasn't", "has not")
    text = text.replace("haven't", "have not")
    text = text.replace("haven't", "have not")
    text = text.replace("haven't", "have not")
    text = text.replace("haven't", "have not")
    text = text.replace("he'd", "he would")
    text = text.replace("haven't", "have not")
    text = text.replace("he'd've", "he would have")
    text = text.replace("'s", "")
    text = text.replace("'t", "")
    text = text.replace("'ve", "")
    text = text.replace(".", " . ")
    text = text.replace("!", " ! ")
    text = text.replace("?", " ? ")
    text = text.replace(";", " ; ")
    text = text.replace(":", " : ")
    text = text.replace("\'", "")
    text = text.replace("\"", "")
    text = text.replace(",", "")
    text = text.replace("[", "")
    text = text.replace("]","")
    text = text.replace("{","")
    text = text.replace("}", "")
    text = text.replace("/", "")
    text = text.replace("|", "")
    text = text.replace("-", "")
    text = text.replace("(", "")
    text = text.replace(")", "")
    text = text.replace("$", "")
    text = text.replace("+", "")
    text = text.replace("*", "")
    text = text.replace("%", "")
    text = text.replace("#", "")
    text = text.lower()
    text = ''.join([i for i in text if not i.isdigit()])

    return text

try:
    
    fables = []
    dirname = os.path.abspath('')
    filepath = os.path.join(dirname, 'input_data/aesopFables.json')

    with open(filepath) as json_file:  
        data = json.load(json_file)
        for p in data['stories']:
            fables.append(' '.join(p['story']))
            
    print('{} fables imported.'.format(len(fables)))
    
    plotsList = fables
    for idx, f in enumerate(fables):
        fables[idx] = clean(f)
    trainPlotsList = fables[:NUM_TRAIN_PLOTS]    

except IOError:
    sys.exit('Cannot find data!')

147 fables imported.


In [4]:
s = 0

for plot in trainPlotsList:
    words = plot.split(' ')
    s += len(words)

mean = s/len(trainPlotsList)
mean

185.7074829931973

__4 - Extract vocabulary__

In [5]:
# CREATE VOCABULARY OF WORDS
idx2word = ['<PAD>','<START>', '<END>']
maxLen = 0
for plot in plotsList:

    words = plot.split(' ')
    
    b=True
    while b:
        if('' in words): 
            words.remove('')
        else: b = False
    
    if len(words) > maxLen : 
        maxLen = len(words)
    
    for word in words:
        if word not in idx2word:
            idx2word.append(word)

word2idx = {}
for word in idx2word:
    word2idx[word] = len(word2idx)

vocabLength = len(idx2word)
print('Vocabulary Size: {}'.format(vocabLength))
print('Fable max length: {}'.format(maxLen))

Vocabulary Size: 3063
Fable max length: 459


__4 - Preprocess__

In [6]:
def createInputTarget(words) :
    
    encoder = []
    decoder = []
    output = []
    
    for i in range(1, len(words)):
        encode_tokens, decode_tokens = words[:i], words[i:]
        encode_tokens = ' '.join(['<START>'] + encode_tokens + ['<END>'])
        output_tokens = ' '.join(decode_tokens + ['<END>'])
        decode_tokens = ' '.join(['<START>'] + decode_tokens + ['<END>'])
        encoder.append(encode_tokens)
        decoder.append(decode_tokens)
        output.append(output_tokens)
        
    return encoder, decoder, output

def getWordTokens(sentence):
    #clean tokens
    words = sentence.split(' ')
    words.append('.')
    b=True
    while b:
        if('' in words): 
            words.remove('')
        else: b = False
    
    return words

def checkMaxLength(words):
    
    seq = []
    
    if len(words) > SENTENCES_MAX_LENGTH :
        seq.append(words[:SENTENCES_MAX_LENGTH])
        seq.append(words[SENTENCES_MAX_LENGTH:])
        while len(seq[-1]) > SENTENCES_MAX_LENGTH:
            tmp = deepcopy(seq[-1])
            del seq[-1]
            seq.append(tmp[:SENTENCES_MAX_LENGTH])
            seq.append(tmp[SENTENCES_MAX_LENGTH:])
    else : 
        seq.append(words)

    return seq

# EXTRACT ENCODER & DECODER INPUT SENTENCES
inputSentences = []
targetSentences = []
outputSentences = []

if PREPROCESS == 'CONTINOUS':
    
    for plot in trainPlotsList:
        words = plot.split(' ')

        b=True
        while b:
            if('' in words): 
                words.remove('')
            else: b = False

        sentences = [words[i:i+SENTENCES_MAX_LENGTH] for i in range(0, len(words), SENTENCES_MAX_LENGTH)]
        for s in sentences:
            for i in range(1, len(s)):
                encode_tokens, decode_tokens = s[:i], s[i:]
                encode_tokens = ' '.join(['<START>'] + encode_tokens + ['<END>'])
                output_tokens = ' '.join(decode_tokens + ['<END>'])
                decode_tokens = ' '.join(['<START>'] + decode_tokens + ['<END>'])
                inputSentences.append(encode_tokens)
                targetSentences.append(decode_tokens)
                outputSentences.append(output_tokens)

elif PREPROCESS == 'DOTS' : 
    
    for plot in trainPlotsList :
        sentences = plot.split('.')
        
        last = None;
        
        for idx, s in enumerate(sentences):
            
            words = getWordTokens(s)
            
            if(len(words) > 2):

                seq = checkMaxLength(words)
                
                for s1 in seq:
                    if(len(s1) > 2):
                        encoder, decoder, output = createInputTarget(s1)
                        inputSentences.extend(encoder)
                        targetSentences.extend(decoder)
                        outputSentences.extend(output)
                
                if(last != None):
                    connection = last[len(last)//2:] + seq[0][:len(seq[0])//2]
                    encoder, decoder, output = createInputTarget(connection)
                    inputSentences.extend(encoder)
                    targetSentences.extend(decoder)
                    outputSentences.extend(output)
                
                last = deepcopy(seq[-1])
                

                
            
numSamples = len(inputSentences)
print('Num samples: {}'.format(numSamples))

print("Creating dataset to feed Model . . . ")
dirname = os.path.abspath('')
filePath = os.path.join(dirname, os.path.join(dirname, 'preprocessed/dataset_fables_{}_{}_{}_{}_{}_{}.csv'.format(
EPOCHS, 
SENTENCES_MAX_LENGTH, 
BATCH_SIZE, 
EMBEDDING_DIM,
HIDDEN_DIM,
NUM_TRAIN_PLOTS)))

if os.path.exists(filePath):
    os.remove(filePath) 

d= {'input_encoder' : inputSentences, 'input_decoder' :targetSentences, 'output_decoder':outputSentences }
df = pd.DataFrame(data=d) 
df = shuffle(df)
df.to_csv(filePath, index=False)

print("Dataset printed on CSV.")

Num samples: 25514
Creating dataset to feed Model . . . 
Dataset printed on CSV.


__5 - Data generator__

In [7]:
def generate_data(word_2_idx, num_samples, max_length, vocab_length, batch_size=BATCH_SIZE):
    '''
    '''
    dirname = os.path.abspath('')
    filePath = os.path.join(dirname, 'preprocessed/dataset_fables_{}_{}_{}_{}_{}_{}.csv'.format(
    EPOCHS, 
    SENTENCES_MAX_LENGTH, 
    BATCH_SIZE, 
    EMBEDDING_DIM,
    HIDDEN_DIM,
    NUM_TRAIN_PLOTS))
    df = pd.read_csv(filePath)
    
    encoderInputData = np.zeros((numSamples, max_length + 2), dtype='int')
    decoderInputData = np.zeros((numSamples, max_length + 2), dtype='int')
    decoderTargetData = np.zeros((numSamples, max_length + 2, 1),dtype='int')
    
    for i in range(0, numSamples):
            
        encoderTokens = df.iloc[[i]]['input_encoder'].values[0].split(' ')
        decoderTokens = df.iloc[[i]]['input_decoder'].values[0].split(' ')
        outputTokens = df.iloc[[i]]['output_decoder'].values[0].split(' ')

        for t, word in enumerate(encoderTokens):
            encoderInputData[i, t] = word_2_idx[word]
        for t, word in enumerate(decoderTokens):
            decoderInputData[i, t] = word_2_idx[word]
        for t, word in enumerate(outputTokens):
            # decoderTargetData is ahead of decoderInputData by one timestep
            decoderTargetData[i, t, 0] = word_2_idx[word]

    
    return encoderInputData, decoderInputData, decoderTargetData

__Extract embeddings matrix__

In [8]:
# Recreating embeddings index based on Tokenizer vocabulary
word2vecModel = gensim.models.Word2Vec.load('embeddings/fables_word2vec_skipgram_128.bin')
word2vec_vocabulary = word2vecModel.wv.vocab
embeddingIndex = dict()
counter = 0
for i, word in enumerate(idx2word):
    if word in word2vec_vocabulary :
        embeddingIndex[word] = word2vecModel[word]
    else:
        counter += 1

print("{} words without pre-trained embedding!".format(counter))
    
# Prepare embeddings matrix
embeddingMatrix = np.random.random((len(word2idx), EMBEDDING_DIM))
for i, word in enumerate(idx2word):
    embeddingVector = embeddingIndex.get(word)
    if embeddingVector is not None:
        embeddingMatrix[i] = embeddingVector

3 words without pre-trained embedding!


/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:8: DeprecationWarning: Call to deprecated `__getitem__` (Method will be removed in 4.0.0, use self.wv.__getitem__() instead).
  


__Or use random weights__

In [ ]:
embeddingMatrix = np.random.random((len(word2idx), EMBEDDING_DIM))

__6 - Train the model__

In [9]:
dirname = os.path.abspath('')

transformerModelPath = os.path.join(dirname, 'models/tr_fables_{}_{}_{}_{}_{}_{}.h5'.format(
    EPOCHS, 
    SENTENCES_MAX_LENGTH, 
    BATCH_SIZE, 
    EMBEDDING_DIM,
    HIDDEN_DIM,
    NUM_TRAIN_PLOTS)
)

# Build the model
model = get_model(
    token_num=len(word2idx),
    embed_dim=EMBEDDING_DIM,
    encoder_num=ENCODERS,
    decoder_num=DECODERS,
    head_num=HEADS_ATTENTION,
    hidden_dim=HIDDEN_DIM,
    attention_activation=ACTIVATION_FUNCTION,
    feed_forward_activation=ACTIVATION_FUNCTION,
    dropout_rate=DROPOUT_RATE,
    embed_weights=embeddingMatrix,
)

model.compile(
    optimizer= keras.optimizers.Adam(),
    loss= keras.losses.sparse_categorical_crossentropy,
    metrics={},
    # Note: There is a bug in keras versions 2.2.3 and 2.2.4 which causes "Incompatible shapes" error, if any type of accuracy metric is used along with sparse_categorical_crossentropy. Use keras<=2.2.2 to use get validation accuracy.
)

model.summary()

if not os.path.exists(transformerModelPath):

    encoderInputData, decoderInputData, decoderTargetData = generate_data(
            word_2_idx=word2idx,
            num_samples=numSamples,
            max_length=SENTENCES_MAX_LENGTH, 
            vocab_length=vocabLength
        )

    # Train the model
    model.fit(
            [encoderInputData, decoderInputData],
            decoderTargetData,
            batch_size=BATCH_SIZE,
            epochs=EPOCHS
            )

    model.save_weights(transformerModelPath) 

else : 
    print('Model already trained')



Instructions for updating:
Colocations handled automatically by placer.
Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
Decoder-Input (InputLayer)      (None, None)         0                                            
__________________________________________________________________________________________________
Encoder-Input (InputLayer)      (None, None)         0                                            
__________________________________________________________________________________________________
Token-Embedding (EmbeddingRet)  [(None, None, 128),  392064      Encoder-Input[0][0]              
                                                                 Decoder-Input[0][0]              
_____________________

25514/25514 [==============================] - 63s 2ms/step - loss: 5.6248
Epoch 7/100
25514/25514 [==============================] - 64s 2ms/step - loss: 5.5987
Epoch 8/100
25514/25514 [==============================] - 63s 2ms/step - loss: 5.5688
Epoch 9/100
25514/25514 [==============================] - 63s 2ms/step - loss: 5.5385
Epoch 10/100
25514/25514 [==============================] - 63s 2ms/step - loss: 5.5054
Epoch 11/100
25514/25514 [==============================] - 63s 2ms/step - loss: 5.4685
Epoch 12/100
25514/25514 [==============================] - 64s 2ms/step - loss: 5.4305
Epoch 13/100
25514/25514 [==============================] - 63s 2ms/step - loss: 5.3938
Epoch 14/100
25514/25514 [==============================] - 63s 2ms/step - loss: 5.3569
Epoch 15/100
25514/25514 [==============================] - 63s 2ms/step - loss: 5.3207
Epoch 16/100
25514/25514 [==============================] - 63s 2ms/step - loss: 5.2824
Epoch 17/100
25514/25514 [======================

25514/25514 [==============================] - 64s 2ms/step - loss: 3.3774
Epoch 100/100
25514/25514 [==============================] - 64s 2ms/step - loss: 3.3639


__7 - Generate sentences__

In [10]:
dirname = os.path.abspath('')

transformerModelPath = os.path.join(dirname, 'models/tr_fables_{}_{}_{}_{}_{}_{}.h5'.format(
    EPOCHS, 
    SENTENCES_MAX_LENGTH, 
    BATCH_SIZE, 
    EMBEDDING_DIM,
    HIDDEN_DIM,
    NUM_TRAIN_PLOTS)
)

# Build the model
model = get_model(
    token_num=len(word2idx),
    embed_dim=EMBEDDING_DIM,
    encoder_num=ENCODERS,
    decoder_num=DECODERS,
    head_num=HEADS_ATTENTION,
    hidden_dim=HIDDEN_DIM,
    attention_activation=ACTIVATION_FUNCTION,
    feed_forward_activation=ACTIVATION_FUNCTION,
    dropout_rate=DROPOUT_RATE,
    embed_weights=embeddingMatrix,
)

model.compile(
    optimizer= keras.optimizers.Adam(),
    loss= keras.losses.sparse_categorical_crossentropy,
    metrics={},
    # Note: There is a bug in keras versions 2.2.3 and 2.2.4 which causes "Incompatible shapes" error, if any type of accuracy metric is used along with sparse_categorical_crossentropy. Use keras<=2.2.2 to use get validation accuracy.
)

model.load_weights(transformerModelPath)

sentences = [
    'The Cock',
    'A Dog and a Wolf',
    'There was once a little Bear', 
    'An eagle was given permission to fly over the country.',
    'A dog was talking to a bear asking for some food. The bear who was hungry too said no.',
    'There was once a little Mouse who walking in the forest. He found his way into a bear cave. It was alone and afraid. The cave was really dark and the Bear was sleeping.'
]

decoded_sentences = []
    
for s in sentences:

    print('Generating from: {}'.format(s))
    encoderTokens = []
    s = clean(s)
    encoderwords = s.split(' ')
    
    b=True
    while b:
        if('' in encoderwords): 
            encoderwords.remove('')
        else: b = False
    
    for w in encoderwords:
        encoderTokens.append(word2idx[w])
    encoderTokens = [word2idx['<START>']] + encoderTokens + [word2idx['<END>']]
    encoderInputData = np.zeros((1, SENTENCES_MAX_LENGTH + 2), dtype='int64')

    decoded = decode(
    model,
    encoderTokens,
    start_token=word2idx['<START>'],
    end_token=word2idx['<END>'],
    pad_token=word2idx['<PAD>'],
    max_len=SENTENCES_MAX_LENGTH,
    )

    decodedPhrase = ''
    for x in decoded:
        decodedPhrase = decodedPhrase + ' ' + idx2word[x]

    decoded_sentences.append(decodedPhrase)
    print('Generated: {}'.format(decodedPhrase))

resultsModelPath = os.path.join(dirname, 'output_data/out_fables_{}_{}_{}_{}_{}_{}.csv'.format(
    EPOCHS, 
    SENTENCES_MAX_LENGTH, 
    BATCH_SIZE, 
    EMBEDDING_DIM,
    HIDDEN_DIM,
    NUM_TRAIN_PLOTS)
)

dict ={
    'phrase' : sentences,
    'generated' : decoded_sentences
}
sentiment_df = pd.DataFrame.from_dict(dict)
sentiment_df.to_csv(resultsModelPath, index=False)

Generating from: The Cock
Generated:  <START> the in the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the dog had been had been had been had been he was very the dog very the dog so the dog as as as as as as as he was to the lion and very the the the the the the the the the the the the the the the the with a plan for and he could not for his claws at as the their and back and back and to to to to his for he along down and to his for . the shepherd his a bird and there to be to be to to to to much answered it the forest he said be axe . i have not to see not is to said the north with the city with a squeal and their their to the train had a succeeded out his frost to children . seeing the mouse in his heels . in his nicely at his his heels again as his grain still
Generating from: A Dog and a Wolf
Generated:  <START> the lion the lion a great . the wolf . the wolf . the wolf . the wolf he was 